In [1]:
%run DataPre-processing.ipynb #importing other jupyter notebook file

In [22]:
import numpy as np

import torch

import random

In [39]:
def data_iter(batch_size, features, labels):
    '''Takes in batch size, feature as np array, labels as np array 
    and returns randomized indices order for picking the features and labels from 
    training example in given batch'''
    
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        #print(batch_indices)
        yield features[batch_indices], labels[batch_indices]

read and print the first small batch of data examples. The shape of the features in each minibatch tells us both the minibatch size and the number of input features. Likewise, our minibatch of labels will have a shape given by batch_size.

As we run the iteration, we obtain distinct minibatches successively until the entire dataset has been exhausted 

In [40]:
batch_size = 10
features = p.x_circle_train_df
#features=  torch.from_numpy(features)
labels = p.y1_circle_train_df 
#labels = torch.from_numpy(labels)

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

[[   0.77524148   81.23314314    2.0140958    12.19506089   -6.1020917
     8.86819816]
 [  21.31020765   71.76650988   34.84167691  -40.9268051   -82.13213403
   -42.33529339]
 [ -27.33712863   -6.2084448    99.9314369    11.97359216  109.14520957
    49.1027787 ]
 [ -17.48468846    1.44743527  -25.99419066   19.52381844  -19.48922123
    -3.50352288]
 [  -5.57724154    2.00152042   35.89886048   37.70467557  -23.51670687
    40.1916273 ]
 [ -63.32160835   58.59941873   11.21804789  -22.27474191  -35.01516576
    65.03184989]
 [ -44.35449417  -33.5509602    43.54610914  -23.80313629  -56.24796648
    37.82530422]
 [  71.56635702   29.0111841   104.1805147   -40.93615752   44.8376818
  -130.58775957]
 [-107.67790985   29.57564535 -113.04509297  -12.40656957  -35.2703956
   -85.80005517]
 [ -34.44056725 -109.12378799   70.90985673  -26.85025078  -82.16158402
    69.83572094]] 
 [[-16.72806317]
 [-24.24449007]
 [ 29.68577723]
 [-33.29769048]
 [  5.84044917]
 [-36.94481261]
 [ -4.66784735

In [ ]:
#Initializing the model parameter

initializing weights by sampling random numbers from a normal distribution with mean 0 and a standard deviation of 0.01, and setting the bias to 0.

In [27]:
w = torch.normal(0, 0.01, size=(6,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

updating them until they fit our data sufficiently well. Each update requires taking the gradient of our loss function with respect to the parameters. Given this gradient, we can update each parameter in the direction that may reduce the loss.


In [42]:
def linreg(X, w, b):  #@save
    return torch.matmul(X, w) + b

Defining the loss function

In [44]:
def squared_loss(y_hat, y):  
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

Optimizing Algorithm

In [46]:
def sgd(params, lr, batch_size):  
    '''Minibatch stochastic gradient descent.'''
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [47]:
#Training
#automatic differentiation, to compute the gradient.
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

TypeError: matmul(): argument 'input' (position 1) must be Tensor, not numpy.ndarray